<a href="https://colab.research.google.com/github/conceptbin/dipstick/blob/master/Dipstick_0_1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Dipstick for Twitter
A simple search and analytics report. Enter your search term and how many tweets you want. Your report (including all the tweets in the dataset) will output to an Excel sheet. This will be saved in the folder where your notebook is located. This project is built on the broad shoulders of [Twint](https://github.com/twintproject/twint), [pandas](https://pandas.pydata.org/docs/index.html#) and [sklearn](https://scikit-learn.org/stable/).

1. Enter your search term and the maximum number of tweets you want collected:

In [0]:
search_term = "Pineapple" #Enter search term here.
limit = 1000 #Change this as you wish. If you're just testing the search, set a smaller limit.

2. In the Runtime-menu, select "Run all". Please be patient while the tweets trickle in.

#Search and save to pandas dataframe

In [0]:
#Install Twint for Twitter search. (This is only necessary if you're using Google Colab)
!pip3 install twint

In [0]:
import pandas as pd
import twint

c = twint.Config()
c.Search = search_term
c.Limit = limit
#c.Min_likes = 5 #Minimum number of likes, to just get tweets people interacted with.
c.Pandas = True

twint.run.Search(c)
df = twint.storage.panda.Tweets_df

#Basic Analysis

##User by frequency

In [0]:
most_tweets = df.groupby(['username']).size().reset_index(name='counts')
most_tweets = most_tweets.sort_values(by='counts', ascending=False)
most_tweets

##Likes

In [0]:
#Most likes
most_l = (df.nlargest(1000, 'nlikes') 
          .drop_duplicates(['tweet'])
          )
most_l = most_l.sort_values(by='nlikes', ascending=False)
most_l[:10][['date','username','tweet','nlikes']]  #Slice of list, selected columns

##Retweets

In [0]:
#Most retweeted
most_r = (df.nlargest(1000, 'nretweets') 
          .drop_duplicates(['tweet'])
          )
most_r[:10][['date','username','tweet','nretweets']]  #Slice of list, selected columns

##Overview

In [0]:
#Overview data
tweets_total = len(df)  #Total no. of tweets in the set
tweeters = len(df['username'].unique())  #No. of unique tweeters
median_likes = df['nlikes'].median() #Median number of likes
median_retweets = df['nretweets'].median()  #Median number of retweets

In [0]:
#Gather overview data into a dict
report = {'What': ['Total no. of tweets in the sample', 'No. of unique tweeters', 'Median likes','Median retweets'],
          'Number': [tweets_total, tweeters, median_likes, median_retweets]}

In [0]:
#Make overview dataframe from dict and display
report_table = pd.DataFrame(report)
report_table

#Keywords

In [0]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

Code below adapted from Susan Li, "A Complete Exploratory Data Analysis and Visualization for Text Data" (Medium, 18 Mar 2019): https://towardsdatascience.com/a-complete-exploratory-data-analysis-and-visualization-for-text-data-29fb1b96fb6a

In [0]:
def preprocess(TweetText):
    TweetText = TweetText.str.replace("(<br/>)", "")
    TweetText = TweetText.str.replace('(<a).*(>).*(</a>)', '')
    TweetText = TweetText.str.replace('(&amp)', '')
    TweetText = TweetText.str.replace('(&gt)', '')
    TweetText = TweetText.str.replace('(&lt)', '')
    TweetText = TweetText.str.replace('(\xa0)', ' ')  
    return TweetText
df['tweet'] = preprocess(df['tweet'])

##Top unigrams before removing stopwords

In [0]:
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_words(df['tweet'], 20)
#for word, freq in common_words:
#    print(word, freq)
df1 = pd.DataFrame(common_words, columns = ['tweet' , 'count'])
#df1.groupby('tweet').sum()['count'].sort_values(ascending=False).iplot(
#   kind='bar', yTitle='Count', linecolor='black', title='Top 20 words in review before removing stop words')

In [0]:
df1

##Top unigrams after removing stop words

In [0]:
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer(stop_words = 'english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_words(df['tweet'], 30)
#for word, freq in common_words:
#    print(word, freq)
df2 = pd.DataFrame(common_words, columns = ['tweet' , 'count'])
df2

# Output

In [0]:
#Save dataframes to separate sheets in an Excel workbook.
with pd.ExcelWriter('dipstick_out.xlsx') as writer:
  report_table.to_excel(writer, sheet_name='Overview report')
  most_l.to_excel(writer, sheet_name='Most likes')
  most_r.to_excel(writer, sheet_name='Most retweets')
  most_tweets.to_excel(writer, sheet_name='Most tweets by user')
  df2.to_excel(writer, sheet_name='Top unigrams (stopwords removed')
  df.to_excel(writer, sheet_name='All tweets')
  
  print("Done!")